# Notebook 2

## Exploração de Dados 2018/2019

### Nelson Costa 42983
### Armando Sousa 76498

## Introdução
O notebook 1 encontra-se dividido em 3 partes: Pré-processamente de dados, modelos preditivos e avaliação dos modelos. Para isto são usados dois data sets diferentes: 'Banknote Authentication Data Set' 
(https://archive.ics.uci.edu/ml/datasets/banknote+authentication) e 'Nursery Data Set' (https://archive.ics.uci.edu/ml/datasets/Nursery). Ambos data sets são usados na parte A e B, enquanto na parte C só é usado o primeiro data set.

### Parte A -> Pré-processamento 
O pré-processamente é um passo importante no inicio de qualquer processo de Data Mining. A transformação de raw information em algo útil.

Começa-se por eliminar entradas nos datasets que tenham dimensões nulas ou vazias. Para os dois data sets escolhidos não existem registos nulos.

### Parte B -> Modelo preditivo 
Os dois modelos escolhidos para treinar foram o MLP e o SVM.


### Parte C -> Avaliação
Como dito no início deste notebook, escolheu-se fazer a avaliação dos modelos onde foi usado o 'Banknote Authentication Data Set'.